# Experiment: Extract Landmarks from Raw Video with MediaPipe Holistic

This notebook demonstrates how to extract pose, face, and hand landmarks from a raw video using MediaPipe Holistic, and save the results in the same .parquet format as the competition dataset.

## 1. Import Required Libraries
Import necessary libraries for video processing, landmark extraction, and saving to .parquet.

In [1]:
# Import Required Libraries
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os

## 2. Set Experiment Parameters
Define input video path, output directory, participant ID, and frame sampling rate.

In [2]:
# Set experiment parameters
input_video_path = "Hailuo_Video_a male person performing sign _434390501941637128.mp4"  # Change to your video file
output_dir = "after/"
participant_id = "demo_user"
sign_label = "demo_sign_after"
frame_sample_rate = 1  # Process every frame (set >1 to skip frames)
os.makedirs(output_dir, exist_ok=True)
output_parquet_path = os.path.join(output_dir, f"{participant_id}_{sign_label}.parquet")

## 3. Load Raw Video
Use OpenCV to load the video and iterate through its frames.

In [3]:
# Load raw video
cap = cv2.VideoCapture(input_video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {frame_count}")

Total frames in video: 141


## 4. Run MediaPipe Holistic Landmark Extraction
Initialize MediaPipe Holistic and process each frame to extract pose, face, and hand landmarks.

In [4]:
# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=2)

landmark_rows = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % frame_sample_rate != 0:
        frame_idx += 1
        continue
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)
    # Helper to extract and append landmarks
    def extract_landmarks(landmarks, ltype):
        if landmarks:
            for idx, lm in enumerate(landmarks.landmark):
                landmark_rows.append({
                    'frame': frame_idx,
                    'type': ltype,
                    'landmark_index': idx,
                    'x': lm.x,
                    'y': lm.y,
                    'z': lm.z,
                    'visibility': getattr(lm, 'visibility', np.nan)
                })
    extract_landmarks(results.face_landmarks, 'face')
    extract_landmarks(results.left_hand_landmarks, 'left_hand')
    extract_landmarks(results.right_hand_landmarks, 'right_hand')
    extract_landmarks(results.pose_landmarks, 'pose')
    frame_idx += 1
cap.release()
holistic.close()
print(f"Total frames processed: {frame_idx}")

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shar

Total frames processed: 141


## 5. Format Extracted Landmarks to DataFrame
Convert the extracted landmarks for each frame into a pandas DataFrame matching the dataset schema.

In [5]:
# Convert to DataFrame
landmarks_df = pd.DataFrame(landmark_rows)
# Add row_id column for uniqueness
landmarks_df['row_id'] = landmarks_df.apply(lambda r: f"{r['frame']}-{r['type']}-{r['landmark_index']}", axis=1)
# Reorder columns to match dataset
landmarks_df = landmarks_df[['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z', 'visibility']]
print(landmarks_df.head())

   frame    row_id  type  landmark_index         x         y         z  \
0      0  0-face-0  face               0  0.493450  0.357994 -0.013742   
1      0  0-face-1  face               1  0.494137  0.320811 -0.029118   
2      0  0-face-2  face               2  0.493934  0.331576 -0.014462   
3      0  0-face-3  face               3  0.490116  0.285871 -0.023331   
4      0  0-face-4  face               4  0.494288  0.309959 -0.031364   

   visibility  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  


## 6. Save Landmarks to .parquet File
Save the DataFrame to a .parquet file using pandas and pyarrow, matching the dataset format.

In [6]:
# Save to .parquet file
landmarks_df.to_parquet(output_parquet_path, index=False)
print(f"Saved extracted landmarks to {output_parquet_path}")

Saved extracted landmarks to after/demo_user_demo_sign_after.parquet
